# Aspects (plus) avancés pour la bioinformatique: recherche dans une base de données distante, version JSON et MyGene.info
Le processus de collecte d'informtation pour un processus d'annotation requiert des recherches sur des bases de données pré-existantes. De manière non-programmatique, on commence par aller sur une page web, on inscrit un ou plusieurs critères de recherche et le site nous retourne un ou des pages de résultats. Ok, c'est cool mais comment intégrer ça dans une structure de données? Il est tout à fait possible de le faire programmatiquement via Python. Dans les exemples qui suivent, nous allons utiliser le service MyGene.info pour faire notre recherche et notre collecte d'informations.
[MyGene.info](https://www.mygene.info) est un service web d'annotation de génomes permettant de faire plusieurs types de recherches et d'obtenir une vaste étendue de données sur ces génomes. Le processus d'utilisation est très souvent le suivant:

 - On utilise un URL spécialement formé pour envoyer une requ&ecirc;te de recherche sur le moteur de recherche et celui-ci retourne un résultat sous la forme d'un flux de texte écrit selon le format [JSON](https://fr.wikipedia.org/wiki/JavaScript_Object_Notation). Ce flux permet la construction d'un dictionnaire que nous pouvons parcourir pour obtenir l'information désirée;
 - Avec cette information, on écrit un 2ème URL spécialement formatté qui nous permettra d'aller spécifiquemment chercher les informations désirées.

Donc, via Python, il faut gérer les communications via URL et construire notre dictionnaire directement en interceptant notre flux JSON. Pour réussier, il nous faut une librairie supplémentaire, `simplejson`, qui permettra de construire automatiquement des dictionnaires à partir du flux JSON.


#### Pré-requis nécessaires

Avant de commencer, assurons-nous uqe les modules nécessaires soient installés sur notre plateforme:

In [1]:
!pip install simplejson

#### Code de démonstration

Voici notre exemple: trouvons des infos sur le gène CTTN chez *H. sapiens*.

In [5]:
# Nous aurons besoin de quelques 
# librairies supplementaires
import simplejson
import requests

# Portion d'URL constant et invariable 
mygene_url = "https://mygene.info/v3/query?"
# Preparation de la requete
headers = {'content-type': 'application/x-www-form-urlencoded'}
params = {
   "q" : "symbol:CTTN",
}
# Envoyez la requete
res = requests.get(mygene_url, params=params, headers=headers)

data = res.json()

# Remarquez que chaque genome qui contient un gene orthologue ou similaire sera
# dans la liste :-)
print(data)

{'took': 4, 'total': 516, 'max_score': 18.06191, 'hits': [{'_id': '2017', '_score': 18.06191, 'entrezgene': '2017', 'name': 'cortactin', 'symbol': 'CTTN', 'taxid': 9606}, {'_id': '13043', '_score': 15.1487, 'entrezgene': '13043', 'name': 'cortactin', 'symbol': 'Cttn', 'taxid': 10090}, {'_id': '60465', '_score': 12.818131, 'entrezgene': '60465', 'name': 'cortactin', 'symbol': 'Cttn', 'taxid': 10116}, {'_id': '113036537', '_score': 11.652846, 'entrezgene': '113036537', 'name': 'cortactin', 'symbol': 'cttn', 'taxid': 8154}, {'_id': '100466994', '_score': 11.652846, 'entrezgene': '100466994', 'name': 'cortactin', 'symbol': 'CTTN', 'taxid': 9646}, {'_id': 'ENSBTAG00000006071', '_score': 11.652846, 'name': 'cortactin', 'symbol': 'CTTN', 'taxid': 9913}, {'_id': '113886595', '_score': 11.652846, 'entrezgene': '113886595', 'name': 'cortactin', 'symbol': 'CTTN', 'taxid': 30522}, {'_id': 'ENSAPEG00000000238', '_score': 11.652846, 'name': 'cortactin', 'symbol': 'CTTN', 'taxid': 161767}, {'_id': '1

Parcourons notre dictionnaire:

In [4]:
for i in data["hits"]:
    print("ID: "+i['_id'])

ID: 2017
ID: 13043
ID: 60465
ID: 113036537
ID: 100466994
ID: ENSBTAG00000006071
ID: 113886595
ID: ENSAPEG00000000238
ID: 105717656
ID: 101802254


Le quel est le bon, celui que l'on recherche? Il est possible de limiter notre recherche en utilisant des champs supplémentaires; dans notre cas, on peut utiliser <i>H. sapiens</i> comme champs limitant:

In [6]:
# Preparation de la requete
headers = {'content-type': 'application/x-www-form-urlencoded'}
mygene_url = "https://mygene.info/v3/query?"

# La valeur 9606 pour le champs species correspond à H. sapiens
params = {
   "q" : "symbol:CTTN",
   "species" : "9606"
}

# Envoyez la requete
res = requests.get(mygene_url, params=params, headers=headers)

data = res.json()

print(data)

{'took': 2, 'total': 1, 'max_score': 18.063349, 'hits': [{'_id': '2017', '_score': 18.063349, 'entrezgene': '2017', 'name': 'cortactin', 'symbol': 'CTTN', 'taxid': 9606}]}


La syntaxte complète des fonctions de recherche est très bien décrite ici: <a href="https://docs.mygene.info/en/latest/doc/query_service.html" target="_blank" rel="noopener">https://docs.mygene.info/en/latest/doc/query_service.html</a>
Celui que nous voulons est la cortactine avec la valeur ID à 2017; c'est d'ailleurs toujours en utilisant la valeur du champs '_id' que le reste se fera ;-) Donc, si votre recherche retourne plusieurs entrées possibles, simplement utiliser une boucle pour parcourir le dictionnaire.
Quoi faire pour la suite? Utilisons la prochaine fonction dans MyGene: l'annotation complète du gène. Encore une fois, on construit un URL spécial pour ce faire, accédant à cette fonction.

In [9]:
# Preparation de la requete
headers = {'content-type': 'application/x-www-form-urlencoded'}
# Portion d'URL constant et invariable
# Cependant, on inscrit statiquement le ID à 2017
mygene_url = "https://mygene.info/v3/gene/2017"

# Envoyez la requete
res = requests.get(mygene_url,headers=headers)

data = res.json()

# Torrent de donnees!!!!
print(data)

{'AllianceGenome': '3338', 'HGNC': '3338', 'MIM': '164765', '_id': '2017', '_version': 7, 'accession': {'genomic': ['AB036705.1', 'AJ288897.2', 'AP000487.6', 'CH471076.1', 'CP068267.2', 'NC_000011.10', 'NC_060935.1', 'NG_029881.1', 'NW_021160005.1'], 'protein': ['AAA58455.1', 'AAH08799.1', 'AAH33889.1', 'ADO22466.1', 'BAD06416.1', 'BAD96333.1', 'BAF83786.1', 'BAG52416.1', 'BAG65370.1', 'CEF49523.1', 'EAW74766.1', 'EAW74767.1', 'EAW74768.1', 'EAW74769.1', 'EAW74770.1', 'NP_001171669.1', 'NP_005222.2', 'NP_612632.1', 'Q14247.2', 'XP_006718510.1', 'XP_006718511.1', 'XP_016872801.1', 'XP_054188449.1', 'XP_054188450.1', 'XP_054188451.1', 'XP_054223937.1', 'XP_054223938.1', 'XP_054223939.1'], 'rna': ['AA325227.1', 'AK023333.1', 'AK091778.1', 'AK222613.1', 'AK291097.1', 'AK304582.1', 'AU124856.1', 'BC008799.2', 'BC033889.1', 'DA716772.1', 'GQ900949.1', 'LN607836.1', 'M98343.1', 'NM_001184740.2', 'NM_005231.4', 'NM_138565.3', 'XM_006718447.4', 'XM_006718448.5', 'XM_017017312.3', 'XM_054332474.

En utilisant seulement le ID, c'est sûr que le résultat sera... impressionnant ;-) On peut utiliser deux approches:

 - Soit on fait la recherche intégrale et on filtre ensuite sur le dictionnaire;

 - Soit on limite le retour d'information en spécifiant uniquement les champs désirés.

Dans l'exemple ci-dessous, les deux méthodes sont faites séquentiellement mais vous verrez que vous aurez le même résultat :-)


In [8]:
# Nous aurons besoin de quelques 
# librairies supplementaires
import simplejson
import requests

# Preparation de la requete
headers = {'content-type': 'application/x-www-form-urlencoded'}

# Methode : construction d'un dictionnaire large
mygene_url = "https://mygene.info/v3/gene/2017"
# Envoyez la requete
res = requests.get(mygene_url,headers=headers)
data = res.json()
print(data['genomic_pos'])
print(data['summary'])

print("=============")

# Methode 2: construction d'un dictionnaire restreint
# Il y a près de 350 champs possibles...
params = {
    "fields" : "genomic_pos,summary"
}

# Envoyez la requete
res = requests.get(mygene_url, params=params,headers=headers)
data = res.json()
print(data['genomic_pos'])
print(data['summary'])

[{'chr': '11', 'end': 70436584, 'ensemblgene': 'ENSG00000085733', 'start': 70398404, 'strand': 1}, {'chr': 'HG2115_PATCH', 'end': 197872, 'ensemblgene': 'ENSG00000288401', 'start': 159692, 'strand': 1}]
This gene is overexpressed in breast cancer and squamous cell carcinomas of the head and neck. The encoded protein is localized in the cytoplasm and in areas of the cell-substratum contacts. This gene has two roles: (1) regulating the interactions between components of adherens-type junctions and (2) organizing the cytoskeleton and cell adhesion structures of epithelia and carcinoma cells. During apoptosis, the encoded protein is degraded in a caspase-dependent manner. The aberrant regulation of this gene contributes to tumor cell invasion and metastasis. Three splice variants that encode different isoforms have been identified for this gene. [provided by RefSeq, May 2010].
[{'chr': '11', 'end': 70436584, 'ensemblgene': 'ENSG00000085733', 'start': 70398404, 'strand': 1}, {'chr': 'HG2115